# Conjuguaison de verbes en francais

## Preparation de donnnée

In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf


from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [20]:
df = pd.read_csv('verbes_conjuges_avec_groupe.csv')
df1 = df.copy()
df1['temps_verbal'] = df1.pop('mode') + ' ' +df1.pop('temps')
df1.rename(columns={'conjugaison': 'verbe_conjugue'}, inplace=True)
df1['verbe_conjugue'] = '<start> ' + df1['verbe_conjugue'] + ' <end>'
df1

,verbe,groupe,personne,verbe_conjugue,temps_verbal
0,vêtir,troisième groupe,pps,<start> jevêts <end>,indicatif présent
1,vêtir,troisième groupe,dps,<start> tuvêts <end>,indicatif présent
2,vêtir,troisième groupe,tps,<start> ilvêt <end>,indicatif présent
3,vêtir,troisième groupe,ppp,<start> nousvêtons <end>,indicatif présent
4,vêtir,troisième groupe,dpp,<start> vousvêtez <end>,indicatif présent
...,...,...,...,...,...
187963,planter,premier groupe,dps,<start> plantons <end>,impératif présent
187964,planter,premier groupe,tps,<start> plantez <end>,impératif présent
187965,planter,premier groupe,pps,<start> aie planté <end>,impératif passé
187966,planter,premier groupe,dps,<start> ayons planté <end>,impératif passé


In [21]:
df1 = df1.sample(n=20000)

In [22]:
# Fusionnons les colonnes d'entrée en une seule chaîne pour simplifier la tokenisation
df1['input'] = df1[['verbe', 'temps_verbal', 'personne']].agg(' '.join, axis=1)

Automation should concerns data processing and training

In [23]:
# Tokenisons les entrées et les sorties
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
tokenizer = Tokenizer(filters=filters)

# Assurez que toutes les données sont des chaînes de caractères
df1['input'] = df1['input'].astype(str)
df1['verbe_conjugue'] = df1['verbe_conjugue'].astype(str)

tokenizer.fit_on_texts(pd.concat([df1['input'], df1['verbe_conjugue']]))
# tokenizer.fit_on_texts(df1['input'])
# tokenizer.fit_on_texts(df1['verbe_conjugue'])

vocab_size = len(tokenizer.word_index) + 1

In [43]:
df1

,verbe,groupe,personne,verbe_conjugue,temps_verbal,input
44200,louer,premier groupe,dpp,<start> vous louerez <end>,indicatif futur simple,louer indicatif futur simple dpp
134948,annuler,premier groupe,tps,<start> qu'il eût annulé <end>,subjonctif plus-que-parfait,annuler subjonctif plus-que-parfait tps
36966,adopter,premier groupe,pps,<start> j'ai adopté <end>,indicatif passé composé,adopter indicatif passé composé pps
53965,tracer,premier groupe,dps,<start> tu traçais <end>,indicatif imparfait,tracer indicatif imparfait dps
3369,exempter,premier groupe,ppp,<start> nous avons exempté <end>,indicatif passé composé,exempter indicatif passé composé ppp
...,...,...,...,...,...,...
35628,relater,premier groupe,pps,<start> je relatais <end>,indicatif imparfait,relater indicatif imparfait pps
47273,arguer,premier groupe,tpp,<start> ils argueront <end>,indicatif futur simple,arguer indicatif futur simple tpp
138279,naître,troisième groupe,ppp,<start> nous naîtrons <end>,indicatif futur simple,naître indicatif futur simple ppp
184788,peinturer,premier groupe,pps,<start> j'eusse peinturé <end>,conditionnel passé deuxième forme,peinturer conditionnel passé deuxième forme pps


In [24]:
# Convertissons les textes en séquences d'entiers
sequence_inputs = tokenizer.texts_to_sequences(df1['input'])
sequence_outputs = tokenizer.texts_to_sequences(df1['verbe_conjugue'])

In [25]:
# Paddons les séquences pour qu'elles aient toutes la même longueur
max_seq_length = max(max(len(seq) for seq in sequence_inputs), max(len(seq) for seq in sequence_outputs))
encoder_input_data = pad_sequences(sequence_inputs, maxlen=max_seq_length, padding='post')
decoder_input_data = pad_sequences(sequence_outputs, maxlen=max_seq_length, padding='post')

In [26]:
# Préparons les données de sortie (décalées d'un pas de temps et catégorisées)
decoder_output_data = np.zeros_like(decoder_input_data)
decoder_output_data[:, 0:-1] = decoder_input_data[:, 1:]
decoder_output_data = to_categorical(decoder_output_data, num_classes=vocab_size)

In [27]:
if False:
    # Séparation initiale des entrées
    inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(
        np.hstack((encoder_input_data, decoder_input_data)),  # Empilage horizontal pour conserver la correspondance
        decoder_output_data,
        test_size=0.2,
        random_state=42
    )

    # Extraire encoder_input et decoder_input des ensembles d'entraînement et de test
    encoder_input_train = inputs_train[:, :encoder_input_data.shape[1]]  # Première moitié des colonnes
    decoder_input_train = inputs_train[:, encoder_input_data.shape[1]:]  # Seconde moitié des colonnes

    encoder_input_test = inputs_test[:, :encoder_input_data.shape[1]]
    decoder_input_test = inputs_test[:, encoder_input_data.shape[1]:]

    #Sauvegarder les outputs
    decoder_target_train = outputs_train
    decoder_target_test = outputs_test

In [28]:
# Séparation des données en ensembles d'entraînement et de test
encoder_input_train, encoder_input_test, decoder_input_train, decoder_input_test, decoder_output_train, decoder_output_test = train_test_split(encoder_input_data, decoder_input_data, decoder_output_data, test_size=0.2, random_state=42)


## Modele encodeur decodeur

In [29]:
import tensorflow as tf

from tensorflow.keras.metrics import Metric

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Embedding, Concatenate, TimeDistributed, Attention

In [38]:
class ExactMatch(Metric):
    def __init__(self, name='exact_match', **kwargs):
        super(ExactMatch, self).__init__(name=name, **kwargs)
        self.correct_predictions = self.add_weight(name='cp', initializer='zeros')
        self.total_predictions = self.add_weight(name='tp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        matches = tf.cast(tf.equal(tf.argmax(y_true, axis=-1), y_pred), 'float32')
        self.correct_predictions.assign_add(tf.reduce_sum(matches))
        # self.total_predictions.assign_add(tf.size(matches))
        self.total_predictions.assign_add(tf.cast(tf.size(matches), 'float32'))

    def result(self):
        return self.correct_predictions / self.total_predictions

    def reset_states(self):
        self.correct_predictions.assign(0.)
        self.total_predictions.assign(0.)

### Avec une chouche de LSTM

In [31]:
if False:

    # Utilisation des valeurs définies lors de la préparation des données
    vocab_size = len(tokenizer.word_index) + 1  # Ajoutez 1 pour le token 0 qui n'est pas utilisé
    max_seq_length = max_seq_length  # Défini lors de la préparation des données

    # Hypothétiques dimensions d'embedding et unités LSTM
    embedding_dim = 256
    lstm_units = 512

    # Encodeur
    encoder_inputs = Input(shape=(max_seq_length,))
    encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Décodeur
    decoder_inputs = Input(shape=(max_seq_length,))
    decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

    # Mécanisme d'attention
    attention_layer = Attention()
    attention_result = attention_layer([decoder_outputs, encoder_outputs])
    decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attention_result])

    # Pour la génération de la sortie
    decoder_dense = TimeDistributed(Dense(vocab_size, activation='softmax'))
    decoder_outputs = decoder_dense(decoder_concat_input)

    # Construction du modèle
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Compilation du modèle
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Résumé du modèle
    model.summary()


### Avec deux couches de LSTM

In [36]:

# Utilisation des valeurs définies lors de la préparation des données
vocab_size = len(tokenizer.word_index) + 1  # Ajoutez 1 pour le token 0 qui n'est pas utilisé
max_seq_length = max_seq_length  # Défini lors de la préparation des données

# Hypothétiques dimensions d'embedding et unités LSTM
embedding_dim = 256
lstm_units = 512

# Encodeur
encoder_inputs = Input(shape=(max_seq_length,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_dropout = Dropout(0.5)(encoder_embedding)  # Appliquez le Dropout à l'embedding
encoder_lstm1 = LSTM(lstm_units, return_sequences=True, return_state=False)(encoder_dropout)
encoder_dropout1 = Dropout(0.5)(encoder_lstm1)  # Appliquez le Dropout après la première couche LSTM
encoder_lstm2 = LSTM(lstm_units, return_sequences=True, return_state=True)(encoder_dropout1)
encoder_outputs, state_h, state_c = encoder_lstm2 
encoder_states = [state_h, state_c]

# Décodeur
decoder_inputs = Input(shape=(max_seq_length,))
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_dropout = Dropout(0.5)(decoder_embedding)  # Appliquez le Dropout à l'embedding
decoder_lstm1 = LSTM(lstm_units, return_sequences=True, return_state=False)(decoder_dropout, initial_state=encoder_states)
decoder_dropout1 = Dropout(0.5)(decoder_lstm1)  # Appliquez le Dropout après la première couche LSTM
decoder_lstm2 = LSTM(lstm_units, return_sequences=True, return_state=False)(decoder_dropout1)  # Pas besoin d'état initial ici puisque c'est déjà passé à decoder_lstm1
decoder_outputs = decoder_lstm2


# Mécanisme d'attention
attention_layer = Attention()
attention_result = attention_layer([decoder_outputs, encoder_outputs])
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attention_result])


# Pour la génération de la sortie
decoder_dense = TimeDistributed(Dense(vocab_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Construction du modèle
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Résumé du modèle
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 embedding_8 (Embedding)     (None, 8, 256)               3305472   ['input_9[0][0]']             
                                                                                                  
 dropout_12 (Dropout)        (None, 8, 256)               0         ['embedding_8[0][0]']         
                                                                                                  
 input_10 (InputLayer)       [(None, 8)]                  0         []                            
                                                                                            

## Entrainement evaluation test

In [ ]:
import matplotlib.pyplot as plt

def train_validation_graph(history):
    # Accéder à l'historique de la perte
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Accéder à l'historique de la précision
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    # Nombre d'époques
    epochs = range(1, len(loss) + 1)

    # Tracé de la perte
    plt.figure(figsize=(14, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='Perte d\'entraînement')
    plt.plot(epochs, val_loss, label='Perte de validation')
    plt.title('Perte d\'entraînement et de validation')
    plt.xlabel('Époques')
    plt.ylabel('Perte')
    plt.legend()

    # Tracé de la précision
    plt.subplot(1, 2, 2)
    # plt.plot(epochs, accuracy, 'bo', label='Précision d\'entraînement')
    # plt.plot(epochs, val_accuracy, 'b', label='Précision de validation')
    plt.plot(epochs, accuracy, label='Précision d\'entraînement')
    plt.plot(epochs, val_accuracy, label='Précision de validation')
    plt.title('Précision d\'entraînement et de validation')
    plt.xlabel('Époques')
    plt.ylabel('Précision')
    plt.legend()

    plt.show()

In [ ]:
def prediction_conjugaison(verbe:str, temps:str, personne:str, model):
    input_text = [' '.join([verbe, temps, personne])]
    print(input_text)
    
    # Tokenisation de l'entrée
    input_seq = tokenizer.texts_to_sequences(input_text)

    # Padding de la séquence pour qu'elle corresponde à la longueur attendue par le modèle
    input_seq_padded = pad_sequences(input_seq, maxlen=max_seq_length, padding='post')

    #Initialisation de la sequnce d'entree du decodeur avec le token de debut
    start_token = tokenizer.word_index['<start>']    
    end_token = tokenizer.word_index['<end>']
    
    
    predicted_seq = []
    for _ in range(max_seq_length):
        old_pred = [start_token]
        for token in predicted_seq:
            if token not in [0, 2]:
                old_pred.append(token)
        # print('old', old_pred)
        decoder_input_seq = np.zeros((1, max_seq_length))
        for index, token in  enumerate(old_pred):
            if token == 2 and token != 0:
                continue
            decoder_input_seq[0, index] = token  # Première position avec le token de debut
        # Prédiction de la séquence de sortie
        # print(decoder_input_seq)
        prediction = model.predict([input_seq_padded, decoder_input_seq])#np.zeros((1, max_seq_length))  # La deuxième partie est le début du décodeur
        predicted_seq = np.argmax(prediction, axis=-1)[0]  # Prendre l'indice avec la probabilité la plus élevée
        # print('predict:', predicted_seq)
    
    # Conversion de la séquence d'entiers en mots
    predicted_text = [tokenizer.index_word[idx] for idx in predicted_seq if (idx > 0) and (idx != start_token) and (idx != end_token)]
    

    # Assemblage du texte prédit
    predicted_conjugation = ' '.join(predicted_text).replace('end', '').strip()
    return predicted_conjugation


### Entrainement

Automation can be done here

In [39]:
# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[ExactMatch()])

# Entraînement du modèle
history = model.fit([encoder_input_train, decoder_input_train], decoder_output_train,
          batch_size=64,
          epochs=15,
          validation_split=0.2)


Epoch 1/5


2024-03-14 11:11:16.817917: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "109" frequency: 2700 num_cores: 4 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 3145728 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


200/200 [==============================] - ETA: 0s - loss: 2.4406 - exact_match: 0.6583

/Users/erictamno/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:2620: UserWarning: Metric ExactMatch implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
2024-03-14 11:22:57.468682: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "109" frequency: 2700 num_cores: 4 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 3145728 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


200/200 [==============================] - 772s 4s/step - loss: 2.4406 - exact_match: 0.6583 - val_loss: 1.7171 - val_exact_match: 0.7143
Epoch 2/5
200/200 [==============================] - 522s 3s/step - loss: 1.5966 - exact_match: 0.7240 - val_loss: 1.5842 - val_exact_match: 0.7516
Epoch 3/5
200/200 [==============================] - 462s 2s/step - loss: 1.3748 - exact_match: 0.7680 - val_loss: 1.4111 - val_exact_match: 0.8040
Epoch 4/5
200/200 [==============================] - 1102s 6s/step - loss: 1.1623 - exact_match: 0.8192 - val_loss: 1.3280 - val_exact_match: 0.8440
Epoch 5/5
200/200 [==============================] - 570s 3s/step - loss: 1.0261 - exact_match: 0.8526 - val_loss: 1.2976 - val_exact_match: 0.8645


In [ ]:
train_validation_graph(history)

### Evaluation

In [40]:
# Évaluation du modèle sur les données de test
model.evaluate([encoder_input_test, decoder_input_test], decoder_output_test)

2024-03-14 12:09:16.213645: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "109" frequency: 2700 num_cores: 4 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 3145728 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


125/125 [==============================] - 89s 634ms/step - loss: 1.3059 - exact_match: 0.8635


[1.3059414625167847, 0.8634687662124634]

### Test

In [64]:
# preparation_prediction('adopter', 'indicatif passé composé', 'pps', model)
prediction_conjugaison('souper', 'indicatif présent', 'ppp', model)

['souper indicatif présent ppp']
1/1 [==============================] - 0s 54ms/step


'nous nous me me skie'

In [41]:
model.save('model_conjugaison.h5')

/Users/erictamno/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
